In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import rich

In [ ]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "o3-mini"

openai = OpenAI()

Advanced reasoning and problem-solving models

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

In [ ]:
prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.chat.completions.create(
    model=MODEL,
    reasoning_effort="medium",
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

print(rich.print(response.usage))
print(response.choices[0].message.content)

CompletionUsage(
    completion_tokens=2236,
    prompt_tokens=44,
    total_tokens=2280,
    completion_tokens_details=CompletionTokensDetails(
        accepted_prediction_tokens=0,
        audio_tokens=0,
        reasoning_tokens=1152,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
)

None
Below is one acceptable solution. This script expects the matrix string as its first (and only) argument. For example, if you run

  ./transpose.sh '[1,2],[3,4],[5,6]'

it will print

  [1,3,5],[2,4,6]

Save the script below as, say, transpose.sh and make it executable.

------------------------------------------------------------
#!/bin/bash
# This script takes a matrix represented as a string in the format
#  "[1,2],[3,4],[5,6]"
# and prints its transpose in the same format.
#
# Usage:
#   ./transpose.sh '[1,2],[3,4],[5,6]'

# Check for input argument
if [ $# -ne 1 ]; then
  echo "Usage: $0 '[1,2],[3,4],[5,6]'"
  exit 1
fi

# Remove any surrounding quotes and then process the string.
# We first remove the leading "[" and trailing "]" from the entire string,
# and then change the row separator "],[" into a semicolon.
input=$(echo "$1" | tr -d "'\"")
rows_str=$(echo "$input" | sed 's/^\[//; s/\]$//; s/\],\[/;/g')

# Split the string into rows.
IFS=";" read -ra rows <<< "$rows_str"

# Responses API

https://platform.openai.com/docs/guides/pdf-files?api-mode=responses

In [ ]:
prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.responses.create(
    model=MODEL,
    reasoning={"effort": "medium"}, # reasoning_effort now become reasoning and accept a dictionary
    input=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

rich.print(response.usage)
print(response.output_text)

ResponseUsage(
    input_tokens=103,
    output_tokens=2175,
    output_tokens_details=OutputTokensDetails(reasoning_tokens=1344),
    total_tokens=2278,
    input_tokens_details={'cached_tokens': 0}
)

Below is one solution. Save the following code as (for example) transpose.sh, then make it executable (chmod +x transpose.sh) and run it by passing the matrix string as an argument.

--------------------------------------------------
#!/bin/bash
# This script takes a matrix represented as a string (e.g., "[1,2],[3,4],[5,6]")
# and prints its transpose in the same format.
#
# Usage:
#   ./transpose.sh "[1,2],[3,4],[5,6]"
#
# Explanation:
#   We first replace the separating “],[” with a semicolon so that
#   the string becomes " [1,2;3,4;5,6] ". Then we remove the leftover outer
#   brackets and split by semicolon to get rows. We extract the elements of
#   each row (split by comma), compute the transpose, and finally print it.

if [ "$#" -ne 1 ]; then
  echo "Usage: $0 '<matrix>'"
  exit 1
fi

matrix="$1"

# Replace "],[" with a semicolon, then remove the first '[' and last ']'
temp="${matrix//],[/\;}"
temp="${temp#[}"
temp="${temp%]}"

# Read the rows into an array (splitting by semico